# IBM:DSP Capstone Project

## Territory Market Development

### Introduction / Business Problem

   
A new technical sales representative for a winery products company has been assigned a new territory. She is to contact and develop sales relationships with wineries in the Pacific Northwestern United States. She seeks to establish new relationships and develop her new sales territory. However, the Pacific Northwestern United States includes both Washington State and Oregon State. The area encompasses vast areas and not all areas are viticulture areas or vineyards. How can she best expand her territory to include new clients?

### Data  

We will use an existing winery list as a substitute for an actual existing client list from data world. We can gather this list by using a web scrape, an SQL query, or pandas. Then, we'll leverage the Foursquare API to create a second list of wineries and vineyards. The second data set will be our potential client data. Once we have current clients and our potential clients we'll separate regions using an unsupervised machine learning algorithm.

The first data set resides at <a href="https://data.world/arthur/wineries">data world</a>. I'll first create an account and attempt to download the file as an excel sheet. This was easy enough to complete. I went into the files and looked around at the data. We are examining the Pacific Northwest Territory so I had to download the file for the <a href="https://query.data.world/s/nib6nc7kfdk7vhbaypnipvdogzcqz2">USA</a>.

❤️🐼❤️ Now that we have downloaded the file from the website, let's look at an alternative using ❤️🐼❤️

In [1]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/r4ahdp3vbrclyyim5siydrszdc6rrx')
df = df.drop(['Unnamed: 3'], axis=1) # Drop the unnamed column
df = df.drop(['Web Site'], axis=1) #Drop web site, come back put in one line
df['lng'] = "" # add Longitude Column
df['lat'] = "" # add Latitude Column
df.rename(columns = {'Winery Name':'Winery'}, inplace = True) #Rename
df.head()

,Winery,State,lng,lat
0,14 Hands,WA,,
1,Abacela Vineyards & Winery,OR,,
2,Abarbanel Wine Co.,NY,,
3,Abbott Winery,CA,,
4,Abeja,WA,,


In [2]:
#To select rows which have WA or OR as the State
states = ['WA','OR']
df = df[df.State.isin(states)]
df.reset_index(drop=True) #Drop the old index

,Winery,State,lng,lat
0,14 Hands,WA,,
1,Abacela Vineyards & Winery,OR,,
2,Abeja,WA,,
3,Academy Wines,OR,,
4,Acme Wineworks,OR,,
...,...,...,...,...
414,Yamhill Valley Vineyards,OR,,
415,Yellow Hawk Cellar,WA,,
416,Youngberg Hill Vineyards,OR,,
417,Zefina Winery,WA,,


In [3]:
#Install Geopy
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# Work the loop, concatenate in.
address = '14 Hands'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

Solving environment: ...working... done

# All requested packages already installed.

-14.694524099999999 -75.11405389565218


### The geocoder is working, kinda, let's (TRY!) to get the loop going and concatenate into our data frame.  

There are some issues here to consider, including the client list will likely be more complete with actual addresses. This was just a winery list and some of the winery names are not registered with the geocoding services, or maybe there is more than one location. Here, we might illustrate more data wrangling techniques or perform a table scrape; but since we have an artificial client list let's move on, we have a lot to do in a week.

In [4]:
for x in range(len(df)):
    try:
       # time.sleep(1) #should I add a delay
        geocode_result = geolocator.geocode(df.Winery[x])
        df['lat'][x] = geocode_result.latitude
        df['lng'][x] = geocode_result.longitude
    except IndexError:
        print("Address was wrong...") # I got some errors here but, I'm just trying to collect enough dummy data to make a client list
    except Exception as e:
        print("Unexpected error occured.", e ) # Forging ahead, we got some dummy data so, all good. #Come back and take a clean scrape if you think it might kill your grade.

Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 2
Unexpected error occured. 3
Unexpected error occured. 5
Unexpected error occured. 6
Unexpected error occured. 7
Unexpected error occured. 9
Unexpected error occured. 10
Unexpected error occured. 11
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 13
Unexpected error occured. 14
Unexpected error occured. 15
Unexpected error occured. 16
Unexpected error occured. 17
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 19
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 21
Unexpected error occured. 22
Unexpected error occured. 23
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 25
Unexpected error occured. 26
Unexpected error occured. 27
Unexpected error occured. 28
Unexpected error occured. 29
Unexpect

Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 262
Unexpected error occured. 263
Unexpected error occured. 264
Unexpected error occured. 265
Unexpected error occured. 266
Unexpected error occured. 267
Unexpected error occured. 268
Unexpected error occured. 269
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 271
Unexpected error occured. 272
Unexpected error occured. 273
Unexpected error occured. 274
Unexpected error occured. 275
Unexpected error occured. 276
Unexpected error occured. 277
Unexpected error occured. 278
Unexpected error occured. 279
Unexpected error occured. 280
Unexpected error occured. 281
Unexpected error occured. 282
Unexpected error occured. 283
Unexpected error occured. 284
Unexpected error occured. 285
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 287
Unexpected error occured. 288
Unexpected error occured. 289
Unex

In [5]:
#Drop the NaN and it won't bother you later. 
nan_value = float("NaN") #Get rid of floating NaN
df.replace("", nan_value, inplace=True) #Any empties too
df.dropna( # Drop 
axis=0, 
how='any',
thresh=None,
subset=None,
inplace=True)
df.reset_index()
print(df)

                       Winery State         lng        lat
0                    14 Hands    WA  -75.114054 -14.694524
4                       Abeja    WA   32.816667   1.766667
8               Academy Wines    OR  -74.249891  40.744971
53              Amavi Cellars    WA -122.141754  47.733092
60             Amity Vineyard    OR -123.174372  45.117032
82                    Animale    WA  -43.110436 -22.907309
90               Antica Terra    OR -123.174372  45.117032
103            Archery Summit    OR -123.047919  45.257348
122         Ashland Vineyards    OR -122.633782  42.179024
146  Badger Mountain Vineyard    WA -119.339468  46.224048
147               Baer Winery    WA -122.152370  47.769629
151  Bainbridge Island Winery    WA -122.518522  47.624707
174           Barnard Griffin    WA  153.017488 -27.280558
197      Bear Creek Vineyards    OR  -65.637609  44.580333
200              Beaux Freres    OR    2.275383  48.818054
229          Benson Vineyards    WA  139.776598 -37.0507

In [6]:
#I dropped these off because it's a client dummy list and it made sense. However, this is only a demonstration and may be a more powerful one when you consider you can leverage the entire planet.
df = df.drop([0,4,8,82,174,197,200,229,242,251,339,358,399])
client_list = df

### Ok, let's see our dummy client list on the map. 

In [7]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

address = 'Portland OR, USA'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

C:\Users\ddycus\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


Solving environment: ...working... done

# All requested packages already installed.

45.5202471 -122.6741949


In [8]:
# Create a map using folium

map_PNW = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=6)
map_PNW


### Client Map (Not Real)

In [9]:
for _, row in df.iterrows():
    label = '{}'.format(
        row.Winery)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row.lat, row.lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(map_PNW) 
map_PNW

### Let's call the businesses around our clients our neighbors.
Then, let's explore some of our neighboring businesses around our client. 


In [10]:
#Making a neighborhood, now each client is a neighborhood or node
neighborhood_name = df.loc[53,'Winery']
neighborhood_latitude = df.loc[53,'lat']
neighborhood_longitude = df.loc[53,'lng']

print("Latitude and longitude of neighborhood '{}' are [{}, {}]".format(
    neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude of neighborhood 'Amavi Cellars' are [47.7330917, -122.1417536]


In [35]:
# Leverage foursquare
# Construct the URL for Foursquare
#You gotta hide this shit bra

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = ''
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

###

Your credentials:
CLIENT_ID: 
CLIENT_SECRET:


In [12]:
# Use the API to explore away from our home cluster.
limit = 100
radius = 500
explore_url_prefix = 'https://api.foursquare.com/v2/venues/explore'
url = '{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    explore_url_prefix, CLIENT_ID, CLIENT_SECRET, VERSION, 
    neighborhood_latitude, neighborhood_longitude, radius, limit)

In [13]:
import requests # Library to handle requests
results = requests.get(url).json() # Get the venues.


In [14]:
# Explore the venues
venues = results['response']['groups'][0]['items']


In [15]:
# Normalize the JSON response
neighborhood_venues = pd.json_normalize(venues)


In [16]:
# Filter out the venue name, category, latitude and logitude.
venue_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
neighborhood_venues = neighborhood_venues.loc[:, venue_columns]
neighborhood_venues.head(5)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Goose Ridge Winery Tasting Room,"[{'id': '4bf58dd8d48988d14b941735', 'name': 'W...",47.731986,-122.140711
1,Village Wines,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",47.731915,-122.140425
2,Alexandria Nicole Cellars,"[{'id': '4bf58dd8d48988d14b941735', 'name': 'W...",47.733069,-122.141644
3,The Commons,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",47.732299,-122.141946
4,Purple Café and Wine Bar,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",47.731720,-122.141946


In [17]:
# Change the column names to just the last part after the '.'.
neighborhood_venues.columns = [column.split(".")[-1] for column in neighborhood_venues.columns]
neighborhood_venues.head(5)

,name,categories,lat,lng
0,Goose Ridge Winery Tasting Room,"[{'id': '4bf58dd8d48988d14b941735', 'name': 'W...",47.731986,-122.140711
1,Village Wines,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",47.731915,-122.140425
2,Alexandria Nicole Cellars,"[{'id': '4bf58dd8d48988d14b941735', 'name': 'W...",47.733069,-122.141644
3,The Commons,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",47.732299,-122.141946
4,Purple Café and Wine Bar,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",47.731720,-122.141946


In [18]:
# Extract the category names from a row.
# We will get the first item in the categories list and then get its name.
def get_category(row):
    categories_list = row['categories']
    if categories_list:
        return categories_list[0]['name']
    return None

In [19]:
# Replace the values in categories column with the first catogory name.
neighborhood_venues['categories'] = neighborhood_venues.apply(get_category, axis=1)
neighborhood_venues.head()

,name,categories,lat,lng
0,Goose Ridge Winery Tasting Room,Winery,47.731986,-122.140711
1,Village Wines,Wine Bar,47.731915,-122.140425
2,Alexandria Nicole Cellars,Winery,47.733069,-122.141644
3,The Commons,Café,47.732299,-122.141946
4,Purple Café and Wine Bar,Wine Bar,47.731720,-122.141946


### Exploring through the neighborhood we find more wineries.
Let's find other wineries which are client neighbors. We could also look for french restaurants near our clients in case we want to take them out, or we could look for vineyards. These locations represent future clients!

In [20]:
neighborhood_venues.head(5)

,name,categories,lat,lng
0,Goose Ridge Winery Tasting Room,Winery,47.731986,-122.140711
1,Village Wines,Wine Bar,47.731915,-122.140425
2,Alexandria Nicole Cellars,Winery,47.733069,-122.141644
3,The Commons,Café,47.732299,-122.141946
4,Purple Café and Wine Bar,Wine Bar,47.731720,-122.141946


In [21]:
isWinery=neighborhood_venues.categories=='Winery'
isVineyard=neighborhood_venues.categories=='Vineyard'
future_clients = neighborhood_venues[isWinery]
future_clients

,name,categories,lat,lng
0,Goose Ridge Winery Tasting Room,Winery,47.731986,-122.140711
2,Alexandria Nicole Cellars,Winery,47.733069,-122.141644
7,Airfield Estate Winery,Winery,47.731944,-122.140510
12,DeLille Carriage House Tasting Room,Winery,47.731648,-122.141416
22,Pepper Bridge Winery,Winery,47.733969,-122.141213


### We can capture all the neighbors to our current clients which are Wineries or Vineyards

In [22]:
#Create an empty list client neighbors
venue_list = []
    
    #For all the clients, create API search request for keyword and category. 
for Winery, lat, lng in zip(df['Winery'], df['lat'], df['lng']):  
    
    
    #Create the API Request URL
    search_url_prefix = 'https://api.foursquare.com/v2/venues/search'
    search_query = 'Winery'
    category_ID  = "4bf58dd8d48988d14b941735,4bf58dd8d48988d1de941735" #Winery or Vineyard
    radius = 100000
    limit = 1000
   
    url = '{}?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
      search_url_prefix, CLIENT_ID, CLIENT_SECRET, VERSION, 
      lat, lng, category_ID, radius, limit)

    # Make the request
    results = requests.get(url).json()
    venues = results['response']['venues']
    # Normalize the JSON response
    dataframe=pd.json_normalize(venues)
    # Rename
    dingdong=dataframe.rename(columns={"location.lat":"lat", "location.lng":"lng"})
    #Append to the client neighbors list   
    venue_list.append(dingdong)
    
    

In [23]:
#make a new loop zipping them all onto one long dataframe
df_list=[]
for x in range(len(venue_list)):
    if (venue_list[x].empty == True) :
        pass
    else:
        name=venue_list[x].name
        lat=venue_list[x].lat
        lng=venue_list[x].lng
        
    dd=pd.DataFrame(list(zip(name,lat,lng)), columns=['name','lat','lng'])
    df_list.append(dd)
    df = pd.concat(df_list)
  

In [24]:
#looped entry of All clients
for _, row in df.iterrows():
    label = '{}'.format(
        row.name)
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker(
        [row.lat, row.lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=True).add_to(map_PNW) 
    
map_PNW

### Machine Learning


<b>DBSCAN</b> stands for Density-based spatial clustering of applications with noise. <b>DBSCAN</b> is a density based clustering non-parametric algorithm: given a set of points in some space, it groups together points that are closely packed together, marking outliers points that lie alone in low-density regions. <b>DBSCAN</b> is one of the most common clustering algorithms and also most cited in scientific literature. 

The reasons I chose <b>DBSCAN</b>:
<ul>
<li><b>DBSCAN</b> does not require one to specify the number of clusters in the data a priori, as opposed to k-means.</li>
<li><b>DBSCAN</b> can find arbitrarily shaped clusters. It can even find a cluster completely surrounded by (but not connected to) a different cluster. Due to the MinPts parameter, the so-called single-link effect (different clusters being connected by a thin line of points) is reduced.</li>
<li><b>DBSCAN</b> has a notion of noise, and is robust to outliers.</li>
<li><b>DBSCAN</b> requires just two parameters and is mostly insensitive to the ordering of the points in the database. (However, points sitting on the edge of two different clusters might swap cluster membership if the ordering of the points is changed, and the cluster assignment is unique only up to isomorphism.)</li>
<li><b>DBSCAN</b> is designed for use with databases that can accelerate region queries, e.g. using an R* tree.
The parameters minPts and ε can be set by a domain expert, if the data is well understood.</li>
</ul>

In [25]:
### Machine Learning: ###

# Prepare the data, renamed client_list above

potential_list=df # rename the df being used above
client_list=client_list.rename(columns={"Winery":"name"}) # then rename columns
client_list=client_list.drop(columns=["State"]) # drop state
df=pd.concat([client_list, potential_list]) # concatenate

# Dataframe should be ready
import numpy as np
import matplotlib.pyplot as plt # 
from sklearn.cluster import DBSCAN # use scikit learn 
from geopy.distance import great_circle

%matplotlib inline



In [26]:
#Create a new map using folium
map_DBSCAN = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=6)
map_DBSCAN

In [27]:
#looped entry of All clients
for _, row in df.iterrows():
    label = '{}'.format(
        row.name)
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker(
        [row.lat, row.lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=True).add_to(map_DBSCAN) 
    
map_DBSCAN


In [28]:
df = df.drop(columns=(["name"]))




In [29]:
df


,lng,lat
53,-122.141754,47.733092
60,-123.174372,45.117032
90,-123.174372,45.117032
103,-123.047919,45.257348
122,-122.633782,42.179024
...,...,...
45,-122.679195,45.624182
46,-121.543654,45.668204
47,-122.678450,45.539752
48,-122.278476,45.400258


In [30]:
# Cluster and see the results
from sklearn.cluster import DBSCAN
from sklearn import metrics

kms_per_radian = 6371.0088
epsilon = 65 / kms_per_radian 

# Run the DBSCAN from sklearn
db = DBSCAN(eps=epsilon, min_samples=5, algorithm='ball_tree', \
            metric='haversine').fit(np.radians(df))

cluster_labels = db.labels_
n_clusters = len(set(cluster_labels))


n_clusters


5

In [31]:
clusters = pd.DataFrame([df[cluster_labels == n] for n in range(-1, n_clusters)])
clusters

,0
0,lng lat 24 -123.935724 45.7...
1,lng lat 53 -122.141754 47...
2,lng lat 60 -123.174372 45...
3,lng lat 146 -119.339468 46...
4,lng lat 11 -121.552243 45.6...
5,"Empty DataFrame Columns: [lng, lat] Index: []"


In [32]:
dataset = pd.DataFrame({'labels':cluster_labels[:]})
dataset.head(25)


,labels
0,0
1,1
2,1
3,1
4,1
5,2
6,0
7,0
8,2
9,2


In [33]:
a = df.reset_index(drop=True)
b = dataset.labels
c = pd.concat([a,b], axis=1)
c





,lng,lat,labels
0,-122.141754,47.733092,0
1,-123.174372,45.117032,1
2,-123.174372,45.117032,1
3,-123.047919,45.257348,1
4,-122.633782,42.179024,1
...,...,...,...
807,-122.679195,45.624182,1
808,-121.543654,45.668204,3
809,-122.678450,45.539752,1
810,-122.278476,45.400258,1


In [34]:
from random import randint
colors = []

for i in range(4):
    colors.append('#%06X' % randint(0, 0xFFFFFF))


from matplotlib import cm
# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))


# add markers to the map
markers_colors = []
for lat, lng, labels, in zip(c['lat'], c['lng'], c['labels']):
    #label = folium.Popup(str(city)+ ','+str(state) + '- Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        color=colors[labels-1],
        fill=True,
        fill_color=colors[labels-1],
        fill_opacity=0.9).add_to(map_DBSCAN)

map_DBSCAN